<a href="https://colab.research.google.com/github/HwangHanJae/Dacon_tutorial/blob/main/Dacon_fiction_writer_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리 로드

In [ ]:
import numpy as np
import pandas as pd
import warnings
import re
warnings.filterwarnings('ignore')

#데이터 불러오기

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/DataScience/Data/fiction_writer/"
train = pd.read_csv(path + "train.csv", encoding='utf-8')
test = pd.read_csv(path + "test.csv", encoding='utf-8')
submission = pd.read_csv(path + "sample_submission.csv", encoding='utf-8')

In [ ]:
train.head()

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3


In [ ]:
test.head()

,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."
2,2,As the lady had stated her intention of scream...
3,3,“And then suddenly in the silence I heard a so...
4,4,His conviction remained unchanged. So far as I...


In [ ]:
submission.head()

,index,0,1,2,3,4
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,2,0,0,0,0,0
3,3,0,0,0,0,0
4,4,0,0,0,0,0


In [ ]:
train.shape, test.shape

((54879, 3), (19617, 2))

In [ ]:
train['author'].unique()

array([3, 2, 1, 4, 0])

# 전처리

In [ ]:
#부호를 제거해주는 함수

#re.sub함수는 첫번째인자를 두번째인자와 교체해준다.
def alpha_num(text):
  return re.sub('[^a-zA-Z0-9]',' ',text)

train['text'] = train['text'].apply(alpha_num)

In [ ]:
# ""와 같은 부호가 사라진 것을 확인할 수 있다.
train.head()


,index,text,author
0,0,He was almost choking There was so much so m...,3
1,1,Your sister asked for it I suppose,2
2,2,She was engaged one day as she walked in per...,1
3,3,The captain was in the porch keeping himself ...,4
4,4,Have mercy gentlemen odin flung up his han...,3


In [ ]:
#불용어 제거해주는 함수
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = stopwords.words('english')

def remove_stopwords(text):
  final_text = []
 
  for i in text.split():                    #text를 공백을 기준으로 나누어 리스트를 만들어줌
    if i.strip().lower() not in stop_words:  #i.strip()함수는 i의 양옆 공백을 제거
      final_text.append(i.strip())
  return " ".join(final_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#전처리 적용 
#모든 문자를 소문자로 변경해주기
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)

In [ ]:
train.head()

,index,text,author
0,0,almost choking much much wanted say strange ex...,3
1,1,sister asked suppose,2
2,2,engaged one day walked perusing jane last lett...,1
3,3,captain porch keeping carefully way treacherou...,4
4,4,mercy gentlemen odin flung hands write anyway ...,3


In [ ]:
test.head()

,index,text
0,0,think one charming young ladies ever met might...
1,1,replied sudden consciousness find cannot ignor...
2,2,lady stated intention screaming course would s...
3,3,suddenly silence heard sound sent heart mouth ...
4,4,conviction remained unchanged far know believe...


In [ ]:
np.array([x for x in train['text']])

array(['almost choking much much wanted say strange exclamations came lips pole gazed fixedly bundle notes hand looked odin evident perplexity',
       'sister asked suppose',
       'engaged one day walked perusing jane last letter dwelling passages proved jane written spirits instead surprised mr odin saw looking odin meeting putting away letter immediately forcing smile said',
       ..., 'sincere well wisher friend sister lucy odin',
       'wanted lend money', 'certainly occurred said yes like'],
      dtype='<U1443')

In [ ]:
#train, test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])

In [ ]:
X_train

array(['almost choking much much wanted say strange exclamations came lips pole gazed fixedly bundle notes hand looked odin evident perplexity',
       'sister asked suppose',
       'engaged one day walked perusing jane last letter dwelling passages proved jane written spirits instead surprised mr odin saw looking odin meeting putting away letter immediately forcing smile said',
       ..., 'sincere well wisher friend sister lucy odin',
       'wanted lend money', 'certainly occurred said yes like'],
      dtype='<U1443')

In [ ]:
X_test

array(['think one charming young ladies ever met might useful work decided genius way witness way preserved agra plan papers father love emotional thing whatever emotional opposed true cold reason place things never marry lest bias judgment',
       'replied sudden consciousness find cannot ignorant goodness owe feel would express could well know orator',
       'lady stated intention screaming course would screamed additional boldness exertion rendered unnecessary hasty knocking door sooner heard mr odin darted much agility wine bottles began dusting great violence matron sharply demanded',
       ...,
       'odin carried thanks odin proceeded happiness lucy excess time reached bartlett buildings able assure mrs odin called next day congratulations never seen spirits life',
       'soon upon odin leaving room mama said odin alarm subject illness cannot conceal sure odin well almost fortnight yet come nothing real indisposition could occasion extraordinary delay else detain norland',


In [ ]:
y_train

array([3, 2, 1, ..., 1, 3, 0])

# 모델링

In [ ]:
import tensorflow as ts
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from tensorflow.keras import Sequential

In [ ]:
#파라미터 설정
vocab_size = 20000
embedding_dim = 16
max_length = 500
padding_type = 'post'

In [ ]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [ ]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding= padding_type, maxlen = max_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

In [ ]:
#가벼운 NLP 모델 생성
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    GlobalAveragePooling1D(),
    Dense(24, activation='relu'),
    Dense(5, activation='softmax')
])

In [ ]:
#compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
#model.summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 16)           320000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                408       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 125       
Total params: 320,533
Trainable params: 320,533
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#fit model
num_epochs= 20
history = model.fit(train_padded, y_train, epochs=num_epochs,verbose=2, validation_split=0.2)

Epoch 1/20
1372/1372 - 10s - loss: 1.5666 - accuracy: 0.2764 - val_loss: 1.5542 - val_accuracy: 0.2774
Epoch 2/20
1372/1372 - 7s - loss: 1.4559 - accuracy: 0.3695 - val_loss: 1.3586 - val_accuracy: 0.4219
Epoch 3/20
1372/1372 - 7s - loss: 1.2777 - accuracy: 0.4668 - val_loss: 1.2345 - val_accuracy: 0.4877
Epoch 4/20
1372/1372 - 7s - loss: 1.1692 - accuracy: 0.5207 - val_loss: 1.1560 - val_accuracy: 0.5304
Epoch 5/20
1372/1372 - 7s - loss: 1.0793 - accuracy: 0.5644 - val_loss: 1.0817 - val_accuracy: 0.5791
Epoch 6/20
1372/1372 - 7s - loss: 0.9890 - accuracy: 0.6119 - val_loss: 1.0074 - val_accuracy: 0.5972
Epoch 7/20
1372/1372 - 7s - loss: 0.9099 - accuracy: 0.6480 - val_loss: 0.9548 - val_accuracy: 0.6288
Epoch 8/20
1372/1372 - 7s - loss: 0.8409 - accuracy: 0.6799 - val_loss: 0.9848 - val_accuracy: 0.6122
Epoch 9/20
1372/1372 - 7s - loss: 0.7810 - accuracy: 0.7073 - val_loss: 0.8622 - val_accuracy: 0.6666
Epoch 10/20
1372/1372 - 7s - loss: 0.7298 - accuracy: 0.7269 - val_loss: 0.8395 -

In [ ]:
#predict values
pred = model.predict_proba(test_padded)

In [ ]:
#submission
submission[['0','1','2','3','4']] = pred
submission.head()

,index,0,1,2,3,4
0,0,0.000109,8.892132e-01,1.077157e-01,2.934160e-03,0.000028
1,1,0.211541,1.497825e-01,3.127408e-01,5.776790e-03,0.320159
2,2,0.999829,5.887616e-05,6.306669e-10,1.510149e-10,0.000112
3,3,0.000007,4.311142e-08,3.068992e-01,7.637627e-08,0.693094
4,4,0.703682,4.943748e-04,5.462918e-04,2.482293e-01,0.047048


In [ ]:
sub_path = "/content/drive/MyDrive/Colab Notebooks/DataScience/My CSV/Dacon/fiction_writer/"
sub_name = 'baseline_submit.csv'
submission.to_csv(sub_path + sub_name, index=False, encoding='utf-8')